In [ ]:
import praw
import pandas as pd
import re


In [46]:
# Set up PRAW with your Reddit API credentials

reddit = praw.Reddit(client_id = "", 
                     client_secret = "",
                     user_agent = "", 
                     username="",
                    password = "")


# Function to extract posts from a subreddit
def extract_subreddit_data(subreddit, limit=100):
    posts = []
    for submission in reddit.subreddit(subreddit).hot(limit=limit):
        print(submission.author)
        # Extract relevant data from each post
        posts.append({
            'author':submission.author,
            'author_flair_text':submission.author_flair_text,
            'created_utc': submission.created_utc,
            'is_self' : submission.is_self,
            'link_flair_text':submission.link_flair_text,
            'name':submission.name,
            'num_comments':submission.num_comments,
            'score':submission.score,
            'selftext':submission.selftext,
            'stickied':submission.stickied,
            'title':submission.title,
            'upvote_ratio':submission.upvote_ratio
        })
    return posts



# Extract data from r/collegeresults
data = extract_subreddit_data('collegeresults')

LinkOFeare
LinkOFeare
microwavablcardboard
Psychological-Yak274
yt1c1l3f
andreativity_
AppointmentSad6173
Jaisetheboss
CompetitiveSerfdom
MertaLin
Public-Dust9717
A_Xueren
aceofspades2707
bribri_briochebun
seven11zzzz
Big_Increase8256
Key_Confectionery
East-Ice-3564
Acrobatic_Werewolf36
MayIspeaktomods
RhiceRune
Prior_Frosting_4961
FIoatz
UnlikelyPoet6344
Datnotguy17
smilingavocados
Haunting-Issue1307
Successful_Egg9313
highschoolsenior24
121mc555
thr0wawae-
Commercial_Ad9237
soccerbill
Somme_Guy
Minute_Champion_3188
Yaoza77
Curejoker
lividbrawler
Valuable-Plum7137
ashitstainisyou
Complete-Echo2399
Lonely-Candle9015
WesternInvestigator3
melifesux
nad1ine
Accomplished-Yak9318
adex1a
Educational_Slice897
RandomGuyHai
Cade_0576-8595-1627
Account5835
tomboys_in_suits
beanbagchairsarecool
sirmartinwagstaff
acute-angle_
None
RubixCube200
arnaud_wang
chrisabulium
Sad_Drink_8239
Accomplished-Cod3675
Stamser
Burneracct6942000
No-Restaurant4609
Outside_Ad_1447
Expensive-Item-4742
Playful_Video_7

In [47]:

# Convert data to a pandas DataFrame for easier processing
df = pd.DataFrame(data)

In [50]:
df = df[~df.stickied]

In [72]:
#TODO: process link_flair_text

,author,author_flair_text,is_self,link_flair_text,name,num_comments,score,selftext,stickied,title,upvote_ratio
2,microwavablcardboard,College Student,True,3.8+|1500+/34+|STEM,t3_18x4ovs,21,37,‼️ this was for the 2022-2023 admissions cycle...,False,Asian girl w/o competitive ECs shotguns becaus...,0.98
3,Psychological-Yak274,None,True,3.8+|1500+/34+|Bus/Fin|International,t3_18x3fno,19,13,Demographics\n\nGender: Male\n\nRace/Ethnicity...,False,International student satisfied but doesnt kno...,0.85
4,yt1c1l3f,None,True,3.8+|1400+/31+|Art/Hum,t3_18x2mfv,2,13,**Demographics**\n\n* Gender: Nonbinary \n* Ra...,False,FGLI Wasian gets into ED School,0.93
5,andreativity_,HS Senior,True,Other|1200+/25+|STEM,t3_18w3zi1,70,149,Kinda just making this if there are any studen...,False,Black girl gets rejected from dream school but...,0.82
6,AppointmentSad6173,None,True,3.8+|1500+/34+|STEM,t3_18w7lj6,34,90,Demographics: \nGender: Female \nRace/Ethnic...,False,fgli girl gets into columbia engineering w/o a...,0.92
...,...,...,...,...,...,...,...,...,...,...,...
95,Fantastic-Essay8184,None,True,3.6+|1500+/34+|Bus/Fin,t3_18jwvza,9,11,**Demographics**\r \n\r \nGender: male\r \n...,False,state school aficionado gets accepted to dream...,1.00
96,ILovGym,None,True,Other|1400+/31+|Art/Hum,t3_18jhnan,36,127,\*\*Demographics\*\*: I am a low-income (when ...,False,"Polyglot, Global South girl gets into Stanford...",0.96
97,Unlucky-Sample-8582,HS Senior,True,3.4+|1500+/34+|Art/Hum,t3_18jed8x,22,72,\n**Demographics**\n\n* Gender: Female\n* Race...,False,Mid white girl gets accepted to dream school!,0.96
98,ShrixGD,None,True,3.8+|1500+/34+|STEM,t3_18jgwlr,12,43,**Demographics**\n\nGender: Male\n\nRace/Ethni...,False,Math major with no math ECs gets into Duke ED,0.96


In [84]:

def extract_demographics(text):
    demographics = {}

    # Define patterns for each demographic category
    patterns = {
        'Gender': r'Gender\s*:\s*([^\n]+)',
        'Race/ethnicity': r'Race/ethnicity\s*:\s*([^\n]+)',
        'Residence': r'Residence\s*:\s*([^\n]+)',
        'Income Bracket': r'Income Bracket\s*:\s*([^\n]+)',
        'Type of School': r'Type of School\s*:\s*([^\n]+)',
        'Hooks': r'Hooks\s*:\s*([^\n]+)'
    }

    # Extract information for each category
    for category, pattern in patterns.items():
        match = re.search(pattern, text)
        demographics[category] = match.group(1).strip() if match else None

    return demographics

# Example usage
demographics_text = "* Gender: Male\n* Race/ethnicity: Asian (chinese)\n* Residence: NY (luckily not nyc)\n* Income Bracket: 200k+\n* Type of School: kinda competitive public\n* Hooks (Recruited Athlete, URM, First-Gen, Geographic, Legacy, etc.): none!"
demographics_info = extract_demographics(demographics_text)
print(demographics_info)


{'Gender': 'Male', 'Race/ethnicity': 'Asian (chinese)', 'Residence': 'NY (luckily not nyc)', 'Income Bracket': '200k+', 'Type of School': 'kinda competitive public', 'Hooks': None}


In [67]:
for i in range(100):
    print(df.selftext.iloc[i][:200])
    print("\n")
    
# need named entity recognition

‼️ this was for the 2022-2023 admissions cycle (I’m class of 2027) ‼️

This is super late considering it's January 2024, but I've wanted to post my results here since I've lurked on A2C and this subre


Demographics

Gender: Male

Race/Ethnicity: Indian 💀

Residence: Tanzania but studying in India.

Income Bracket: Middlish income, can afford like 50-55k for Uni.  Seeking Aid 💀

Type of School: Inter


**Demographics**

* Gender: Nonbinary 
* Race/Ethnicity: White + Asian (Filipino)
* Residence: MS 
* Income Bracket: ~30k but I have a super complicated economic background because I have legal guardi


Kinda just making this if there are any students like me wondering whether they could actually get into college with "average" stats.

Demographics

Gender: Female

Race/Ethnicity: African American (1


Demographics:  
Gender: Female  
Race/Ethnicity: Jewish, White/Middle Eastern  
Residence: South Florida  
Income Bracket: <45k  
Type of School: Mid Private School (on full scholarship)  

IndexError: single positional indexer is out-of-bounds

In [88]:

# Assuming `post_content` contains the text of the Reddit post
post_content = df.selftext.loc[35]

# Parsing each section
demographics = parse_key_value_section(extract_section(post_content, "Demographics"))

print(demographics)

post_content[:1000]

{'* Gender': 'Male\n* Race/ethnicity: Asian (chinese)\n* Residence: NY (luckily not nyc)\n* Income Bracket: 200k+\n* Type of School: kinda competitive public\n* Hooks (Recruited Athlete, URM, First-Gen, Geographic, Legacy, etc.): none!'}


"**Demographics**\n\n* Gender: Male\n* Race/ethnicity: Asian (chinese)\n* Residence: NY (luckily not nyc)\n* Income Bracket: 200k+\n* Type of School: kinda competitive public\n* Hooks (Recruited Athlete, URM, First-Gen, Geographic, Legacy, etc.): none!\n\n**Intended Major(s):** applied Bioengineering/Biomedical Engineering to all schools\n\n**Academics**\n\n* GPA (UW/W) 98.53 UW/103.17 W. Our school grades out of 100 to be quirky idk\n* Rank (or percentile): school doesn't officially rank, but I am top 1-2% out of \\~350 students\n* Honors/AP/IB/Dual Enrollment/etc.: 9 APs by graduation (Chinese, World History, Computer Science A, US History, Chemistry, Calc BC already taken). 6 DE courses (Syracuse University & local community college). 12 Honors courses\n* Senior year coarse load: AP French, AP Lit, AP Physics C, DE Economics, DE Public policy, Symphony orchestra, Multivariable calculus (online course, school doesn't offer math past calc bc)\n\n**Standardized Testing**\n\n* SAT: 1540

## Demographics

Gender: Male

Race/Ethnicity: asian

Residence: east coast

Income Bracket: ~125k

Type of School: small parochial, 20 in grad class

Hooks (Recruited Athlete, URM, First-Gen, Geographic, Legacy, etc.): none!

Intended Major(s): polisci/public policy (pton’s SPIA)

Academics

GPA (UW/W): 4.13/4.33 (my school does it out of 4.33)

Rank (or percentile): doesn’t rank

/# of Honors/AP/IB/Dual Enrollment/etc.: none! Because my school is weird we all take the same classes (except for math) so no ap/ib/de classes

Senior Year Course Load: multivariable calc (we don’t actually do this but my school labels it this), spanish, theology, choir, theatre, humane letters 4, physics-matlab

Standardized TestingSAT I: 1560 superscore; 770 RW, 790M


In [94]:
import re

def extract_section(post, section_title):
    # Creating a pattern that is case-insensitive
    pattern = r"\*\*{}.*?\*\*(.*?)(?=\*\*[A-Z]|$)".format(re.escape(section_title))
    match = re.search(pattern, post, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else None


def parse_key_value_section(section_text):
    data = {}
    for line in section_text.split('\n\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            data[key.strip()] = value.strip()
    return data

def parse_list_section(section_text):
    items = re.findall(r"\\#(\d+)\s*(.*)", section_text)
    return {f"#{num}": item.strip() for num, item in items}

# Assuming `post_content` contains the text of the Reddit post
post_content = df.selftext.loc[35]

# Parsing each section
demographics = extract_demographics(extract_section(post_content, "Demographics"))
academics = parse_key_value_section(extract_section(post_content, "Academics"))
standardized_testing = parse_key_value_section(extract_section(post_content, "Standardized Testing"))
extracurriculars = parse_list_section(extract_section(post_content, "Extracurriculars/Activities"))
awards = parse_list_section(extract_section(post_content, "Awards/Honors"))
letters_of_recommendation = parse_key_value_section(extract_section(post_content, "Letters of Recommendation"))
essays = parse_key_value_section(extract_section(post_content, "Essays"))
decisions = parse_key_value_section(extract_section(post_content, "Decisions"))

# Output the results
print("Demographics:", demographics)
print("Academics:", academics.keys())
print("Standardized Testing:", standardized_testing)
print("Extracurriculars:", extracurriculars)
print("Awards:", awards)
print("Letters of Recommendation:", letters_of_recommendation)
print("Essays:", essays)
print("Decisions:", decisions)


Demographics: {'Gender': 'Male', 'Race/ethnicity': 'Asian (chinese)', 'Residence': 'NY (luckily not nyc)', 'Income Bracket': '200k+', 'Type of School': 'kinda competitive public', 'Hooks': None}
Academics: dict_keys(['* GPA (UW/W) 98.53 UW/103.17 W. Our school grades out of 100 to be quirky idk\n* Rank (or percentile)'])
Standardized Testing: {'* SAT': '1540 (740 EBRW, 800M)\n* APs: All 5s except a 4 on APUSH. That class was the death of me but its all good'}
Extracurriculars: {}
Awards: {}
Letters of Recommendation: {}
Essays: {'* Common App': 'wrote about growth and change of perspective on life after death of friends. won\'t go into it more because its personal, but I did a lot of introspection during the writing process, I think it is something that I can be proud of.  My AO sent me an email later that said she thought my personal statement was very moving, so I think this essay was a major reason as to why I got in  \n* Penn supplementals: I am proud of them, but they are probably

In [96]:
import re

def extract_section(post, section_title):
    pattern = r"\*\*{}.*?\*\*(.*?)(?=\*\*[A-Z]|$)".format(re.escape(section_title))
    match = re.search(pattern, post, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else None

def parse_key_value_list(section_text):
    items = re.findall(r"\*\s*(.*?):\s*(.*)", section_text)
    return {item[0].strip(): item[1].strip() for item in items}

def parse_numbered_list(section_text):
    items = re.findall(r"^(\d+)\.\s*(.*?)$", section_text, re.MULTILINE)
    return {f"#{num}": item.strip() for num, item in items}

def parse_simple_list(section_text):
    items = re.findall(r"\*\s*(.*)", section_text)
    return items

def parse_academics(post):
    section = extract_section(post, "Academics")
    return parse_key_value_list(section) if section else {}

def parse_standardized_testing(post):
    section = extract_section(post, "Standardized Testing")
    return parse_key_value_list(section) if section else {}

def parse_extracurriculars(post):
    section = extract_section(post, "Extracurriculars/Activities")
    return parse_numbered_list(section) if section else {}

def parse_awards(post):
    section = extract_section(post, "Awards/honors")
    return parse_numbered_list(section) if section else {}

def parse_letters_of_recommendation(post):
    section = extract_section(post, "Letters of Recommendation")
    return parse_simple_list(section) if section else {}

def parse_essays(post):
    section = extract_section(post, "Essays")
    return parse_key_value_list(section) if section else {}

def parse_decisions(post):
    section = extract_section(post, "Decisions")
    return parse_key_value_list(section) if section else {}

# Example usage

academics = parse_academics(post_content)
standardized_testing = parse_standardized_testing(post_content)
extracurriculars = parse_extracurriculars(post_content)
awards = parse_awards(post_content)
letters_of_recommendation = parse_letters_of_recommendation(post_content)
essays = parse_essays(post_content)
decisions = parse_decisions(post_content)

# Output the results
print("Academics:", academics.keys())
print("Standardized Testing:", standardized_testing.keys())
print("Extracurriculars:", extracurriculars)
print("Awards:", awards)
print("Letters of Recommendation:", letters_of_recommendation)
print("Essays:", essays)
print("Decisions:", decisions)


Academics: dict_keys(['Rank (or percentile)', 'Honors/AP/IB/Dual Enrollment/etc.', 'Senior year coarse load'])
Standardized Testing: dict_keys(['SAT', 'APs'])
Extracurriculars: {'#1': ''}
Awards: {'#1': 'Science Olympiad big invitational medal', '#2': 'Science Olympiad states medal', '#3': 'Science Olympiad states medal', '#4': 'National Merit Semifinalist (barely on the cutoff)', '#5': 'AP Scholar with distinction'}
Letters of Recommendation: ['']
Essays: {'Common App': "wrote about growth and change of perspective on life after death of friends. won't go into it more because its personal, but I did a lot of introspection during the writing process, I think it is something that I can be proud of.  My AO sent me an email later that said she thought my personal statement was very moving, so I think this essay was a major reason as to why I got in", 'Penn supplementals': 'I am proud of them, but they are probably very standard essays. I pretty much told them straight up what my life at p

In [90]:
post_content

'**Demographics**\n\n* Gender: Male\n* Race/ethnicity: Asian (chinese)\n* Residence: NY (luckily not nyc)\n* Income Bracket: 200k+\n* Type of School: kinda competitive public\n* Hooks (Recruited Athlete, URM, First-Gen, Geographic, Legacy, etc.): none!\n\n**Intended Major(s):** applied Bioengineering/Biomedical Engineering to all schools\n\n**Academics**\n\n* GPA (UW/W) 98.53 UW/103.17 W. Our school grades out of 100 to be quirky idk\n* Rank (or percentile): school doesn\'t officially rank, but I am top 1-2% out of \\~350 students\n* Honors/AP/IB/Dual Enrollment/etc.: 9 APs by graduation (Chinese, World History, Computer Science A, US History, Chemistry, Calc BC already taken). 6 DE courses (Syracuse University & local community college). 12 Honors courses\n* Senior year coarse load: AP French, AP Lit, AP Physics C, DE Economics, DE Public policy, Symphony orchestra, Multivariable calculus (online course, school doesn\'t offer math past calc bc)\n\n**Standardized Testing**\n\n* SAT: 15

In [5]:
# Data processing steps (e.g., parsing text, extracting features)
# This is an example and needs to be adapted based on actual data format
df['GPA'] = df['selftext'].apply(extract_gpa)  # You need to write the extract_gpa function
df['TestScores'] = df['selftext'].apply(extract_test_scores)  # Similarly for extract_test_scores
# Add more processing as needed

NameError: name 'extract_gpa' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split data into features (X) and target (y)
X = df[['GPA', 'TestScores']]  # Include other features as necessary
y = df['Accepted']  # You need a target variable indicating acceptance

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model (Random Forest as an example)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))


In [ ]:
def get_user_input():
    # Get user data (GPA, Test Scores, etc.)
    # For example:
    user_gpa = float(input("Enter your GPA: "))
    user_test_scores = float(input("Enter your test scores: "))
    return [user_gpa, user_test_scores]

def predict_acceptance(model, user_input):
    # Predict using the model
    prediction = model.predict([user_input])
    return prediction

# Example usage
user_input = get_user_input()
prediction = predict_acceptance(model, user_input)
print("Probability of acceptance:", prediction)
